# Introduction

We intend to produce a model based on BERT to infer capacity of predicting if a tweet can be categorized as an Alert tweet or not.

Some kernels have already implemented such models :

egortrushin/nlp-with-disaster-tweets-roberta-using-pytorch         NLP with Disaster Tweets: RoBERTa using PyTorch     Egor Trushin

ashishsingh226/disaster-tweet-classification-using-distil-bert     Disaster Tweet Classification using Distil Bert     AshishSingh226


We propose another implementation for BERT with fast-bert api on PyTorch

# Imports

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

!pip install fast-bert
from fast_bert.data_cls import BertDataBunch
from fast_bert.learner_cls import BertLearner
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner
from fast_bert.metrics import fbeta, roc_auc
from fast_bert.prediction import BertClassificationPredictor
from pathlib import Path
import pandas as pd
import logging
import os
import pickle
from time import time

logger = logging.getLogger()
device_cuda = torch.device("cuda")

# Dataset loading

In [2]:
data = pd.read_csv('./../dataset/train.csv',sep=",")

In [3]:
data.shape

(7613, 5)

In [4]:
data.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [5]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


# Data cleaning

# Exploratory analysis

# Model